In [10]:
from SimObject import SimObject
from CoinSimulator import CoinSimulator
from CoinSimulator import Coin
from CoinSimulator import CoinFlipSimulator
import sys
import logging
logging.basicConfig(filename=f"{__name__}.log", level=logging.ERROR, format='%(asctime)s - %(levelname)s - %(message)s')
def sim_test(test_id="TEST"):
    logging.info(f"\n### coins ### {test_id} ######################")
def is_positive_int(value):
    try:
        num = int(value)
        return num > 0
    except ValueError:
        return False

def coin_simulator(number_of_bits=1, number_of_tests=10):
    sim_test("COIN SIMULATOR")
    results = CoinSimulator(number_of_bits).simulate(number_of_tests)
    logging.debug(f"Results: {results}")
    return results

def coin_flip_simulator(number_of_coins=1, number_of_tests=10):
    sim_test("COIN FLIP SIMULATOR")
    sim = CoinFlipSimulator()
    results = {}
    for i in range(0, number_of_coins):
        c = Coin("HEADS",f"coin-{i}")
        for state, state_object in c.get_states().items():
            sim.add_event(state_object[SimObject.STATE], state_object[SimObject.PROBABILITY])
        results[c.NAME] = sim.simulate(number_of_tests)
    logging.debug(f"Results: {results}")
    return results

if __name__ == "__main__":
    sim_test("ARGUMENTS")
    logging.info(f"Number of arguments: {len(sys.argv)}")
    logging.info(f"Argument List: {sys.argv}")

    try:
        logging.debug("sys.argv[1]:"+sys.argv[1])
        logging.debug("sys.argv[2]:"+sys.argv[2])
        NUMBER_OF_OBJECTS = int(sys.argv[1])
        NUMBER_OF_SIMULATIONS = int(sys.argv[2])
    except:
        NUMBER_OF_OBJECTS = 3
        NUMBER_OF_SIMULATIONS = 10
    logging.info(f"NUMBER_OF_OBJECTS: {NUMBER_OF_OBJECTS}")
    logging.info(f"NUMBER_OF_SIMULATIONS: {NUMBER_OF_SIMULATIONS}")




In [11]:
for state, result in coin_simulator(NUMBER_OF_OBJECTS,NUMBER_OF_SIMULATIONS).items():
    print(f"state: {state} result: {result}")

state: coin-0-HEADS result: 5
state: coin-0-TAILS result: 5
state: coin-1-HEADS result: 4
state: coin-1-TAILS result: 6
state: coin-2-HEADS result: 5
state: coin-2-TAILS result: 5


In [18]:
NUMBER_OF_OBJECTS = 3
NUMBER_OF_SIMULATIONS = 1000
for coin, result in coin_flip_simulator(NUMBER_OF_OBJECTS,NUMBER_OF_SIMULATIONS).items():
    print(f"coin: {coin}")
    for k,v in result.items():
        print(f"\t{k}:{v}")



coin: coin-0
	heads:502
	tails:498
coin: coin-1
	heads:483
	tails:517
coin: coin-2
	heads:513
	tails:487


In [78]:
from ProbabilisticSimulator import ProbabilisticSimulator

def simulate_object_probability(NUMBER_OF_OBJECTS=1,NUMBER_OF_SIMULATIONS=100):
    sim = ProbabilisticSimulator()
    results = {}
    for i in range(0, NUMBER_OF_OBJECTS):
        o = [
            {"probability":.10},
            {"probability":.20},
            {"probability":.15},
            {"probability":.12},
            {"probability":.13},
            {"probability":.02},
            {"probability":.06},
            {"probability":.22}]
        
        for ndx, state_object in enumerate(o):
            probability = state_object["probability"]
            sim.add_event(f"state-{ndx}-{100*probability}%", probability)
        results[f"object-{i}"] = sim.simulate(NUMBER_OF_SIMULATIONS)
    for item,result in results.items():
        print(f"{item} ran {NUMBER_OF_SIMULATIONS} of trials over {len(o)} states")
        for k,v in result.items():
            v = f"{round(100*(v/NUMBER_OF_SIMULATIONS), len(str(NUMBER_OF_SIMULATIONS))-1)}%"
            print(f"\t{k}\t{v}")

NUMBER_OF_OBJECTS = 5
NUMBER_OF_SIMULATIONS = 1000000
simulate_object_probability()


object-0 ran 100 of trials over 8 states
	state-0-10.0%	9.0%
	state-1-20.0%	18.0%
	state-2-15.0%	19.0%
	state-3-12.0%	14.0%
	state-4-13.0%	11.0%
	state-5-2.0%	6.0%
	state-6-6.0%	4.0%
	state-7-22.0%	19.0%
